#### Import libraries

In [15]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [21]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

#### Make detections

In [26]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Detect stuff and render
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detections
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections
        mp_drawing.draw_landmarks(image,
                                  results.pose_landmarks,
                                  mp_pose.POSE_CONNECTIONS)

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1741934429.984768   22976 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741934429.986278   25708 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) Arc(tm) Graphics (MTL)
W0000 00:00:1741934430.026812   25684 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741934430.046197   25686 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


#### Determining the landmarks

In [31]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Detect stuff and render
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detections
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass

        # Render detections
        mp_drawing.draw_landmarks(image,
                                  results.pose_landmarks,
                                  mp_pose.POSE_CONNECTIONS)

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

I0000 00:00:1741934893.968840   22976 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741934893.970892   26606 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) Arc(tm) Graphics (MTL)
W0000 00:00:1741934894.010186   26583 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741934894.035419   26600 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [42]:
import cv2
import mediapipe as mp
import numpy as np

# Khởi tạo Mediapipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Hàm tính góc giữa hai vector
def angle_between_vectors(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    cos_theta = dot_product / (norm_v1 * norm_v2)
    angle_rad = np.arccos(np.clip(cos_theta, -1.0, 1.0))  # Đảm bảo nằm trong khoảng [-1, 1]
    angle_deg = np.degrees(angle_rad)
    return angle_deg

# Mở camera
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect stuff and render
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Make detections
    results = pose.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Extract shoulder, elbow, wrist landmarks (right)
        shoulder_right = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x,
                                   landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y,
                                   landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].z])

        elbow_right = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y,
                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].z])

        wrist_right = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y,
                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].z])

        # Extract shoulder, elbow, wrist landmarks (left)
        shoulder_left = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x,
                                   landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y,
                                   landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].z])

        elbow_left = np.array([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].z])

        wrist_left = np.array([landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST].z])

        # Vectorization
        upper_arm_right = elbow_right - shoulder_right
        forearm_right = wrist_right - elbow_right
        upper_arm_left = elbow_left - shoulder_left
        forearm_left = wrist_left - elbow_left

        # Angle between 2 vectors
        angle_hand_right = angle_between_vectors(upper_arm_right, forearm_right)
        angle_hand_left = angle_between_vectors(upper_arm_left, forearm_left)

        # Render detections
        mp_drawing.draw_landmarks(image,
                                  results.pose_landmarks,
                                  mp_pose.POSE_CONNECTIONS)

        # Verify if hand is up
        if ((angle_hand_right < 100 and wrist_right[1] < elbow_right[1]) or
            (angle_hand_left < 100 and wrist_left[1] < elbow_left[1])):
            cv2.putText(image, "Hand up", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


    cv2.imshow("Pose Detection", image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1741936168.111689   22976 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741936168.114108   33515 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) Arc(tm) Graphics (MTL)
W0000 00:00:1741936168.156529   33494 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741936168.182447   33507 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [55]:
import cv2
import mediapipe as mp
import numpy as np
from ultralytics import YOLO

# Mediapipe Pose & Hands
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose()
hands = mp_hands.Hands()

# Model YOLO
model = YOLO("yolo-Weights/yolov8n.pt")

# Chỉ nhận diện "person" và "cell phone"
target_classes = {"person", "cell phone"}

# Khởi động webcam
cap = cv2.VideoCapture(0)


def calculate_angle(a, b, c):
    """Tính góc giữa 2 vector a-b và c-b"""
    vector1 = np.array(a) - np.array(b)
    vector2 = np.array(c) - np.array(b)

    unit_vector1 = vector1 / np.linalg.norm(vector1)
    unit_vector2 = vector2 / np.linalg.norm(vector2)

    dot_product = np.dot(unit_vector1, unit_vector2)
    angle = np.arccos(dot_product)  # Radian
    return np.degrees(angle)  # Chuyển sang độ


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Chuyển đổi màu để xử lý với Mediapipe
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    pose_results = pose.process(image)
    hand_results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Nhận diện vật thể bằng YOLO
    yolo_results = model(frame, stream=True)

    # Lưu trữ danh sách vật thể
    detected_objects = {"cell phone": []}

    for r in yolo_results:
        for box in r.boxes:
            cls_name = model.names[int(box.cls[0])]
            if cls_name in target_classes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Toạ độ hộp
                detected_objects.setdefault(cls_name, []).append((x1, y1, x2, y2))
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 3)
                cv2.putText(frame, cls_name, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Xử lý Pose Detection
    if pose_results.pose_landmarks:
        landmarks = pose_results.pose_landmarks.landmark

        def get_point(landmark):
            """Lấy tọa độ (x, y) của landmark."""
            return int(landmarks[landmark].x * frame.shape[1]), int(landmarks[landmark].y * frame.shape[0])

        # Lấy tọa độ vai, khuỷu tay, cổ tay
        shoulder_r, elbow_r, wrist_r = get_point(mp_pose.PoseLandmark.RIGHT_SHOULDER), \
                                       get_point(mp_pose.PoseLandmark.RIGHT_ELBOW), \
                                       get_point(mp_pose.PoseLandmark.RIGHT_WRIST)

        shoulder_l, elbow_l, wrist_l = get_point(mp_pose.PoseLandmark.LEFT_SHOULDER), \
                                       get_point(mp_pose.PoseLandmark.LEFT_ELBOW), \
                                       get_point(mp_pose.PoseLandmark.LEFT_WRIST)

        # Kiểm tra điều kiện giơ tay
        angle_right = calculate_angle(shoulder_r, elbow_r, wrist_r)
        angle_left = calculate_angle(shoulder_l, elbow_l, wrist_l)

        hand_up = ((angle_right < 120 and wrist_r[1] < elbow_r[1]) or
                   (angle_left < 120 and wrist_l[1] < elbow_l[1]))

        # Kiểm tra cầm điện thoại bằng tọa độ bàn tay
        holding_phone = False
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                for i, landmark in enumerate(hand_landmarks.landmark):
                    x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                    # Kiểm tra nếu điểm bàn tay nằm trong vùng điện thoại
                    for phone_x1, phone_y1, phone_x2, phone_y2 in detected_objects["cell phone"]:
                        if phone_x1 <= x <= phone_x2 and phone_y1 <= y <= phone_y2:
                            holding_phone = True
                            break
                    if holding_phone:
                        break

        # Hiển thị kết quả
        if hand_up:
            text, color = ("Hand up", (0, 255, 0)) if not holding_phone else ("Holding phone", (0, 0, 255))
            cv2.putText(frame, text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

        # Vẽ pose landmarks
        mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Vẽ bàn tay
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Hiển thị frame
    cv2.imshow("Pose Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1741943552.885832   22976 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741943552.887872   42364 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) Arc(tm) Graphics (MTL)
I0000 00:00:1741943552.892468   22976 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741943552.895760   42389 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: Mesa Intel(R) Arc(tm) Graphics (MTL)
W0000 00:00:1741943552.918313   42365 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741943552.928890   42344 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741943552.932218   42373 inference_feedback_manager.cc:114] Feedback mana


0: 480x640 1 person, 1 toothbrush, 3.3ms
Speed: 0.9ms preprocess, 3.3ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 4.0ms
Speed: 1.1ms preprocess, 4.0ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 3.9ms
Speed: 1.0ms preprocess, 3.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 5.3ms
Speed: 1.0ms preprocess, 5.3ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 3.7ms
Speed: 1.0ms preprocess, 3.7ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 3.6ms
Speed: 1.0ms preprocess, 3.6ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 5.7ms
Speed: 2.0ms preprocess, 5.7ms inference, 1.1ms postprocess per image at shape (1, 3, 480,